In [ ]:
import ipywidgets as widgets
import pandas as pd
import sqlalchemy as sa

import pathlib

from IPython.display import display, HTML

In [4]:
cwd = pathlib.Path.cwd()
folder_path = cwd
#[f for f in folder_path.glob('*/*')]

WindowsPath('C:/Users/DominicHunt/OneDrive - Corndel Ltd/Projects/Pyodide')

In [5]:
#with open(folder_path / "Pagilla ERD v3.png", "rb") as file:
#    image = file.read()
#widgets.Image(
#    value=image,
#    format='png',
#    width=900,
#    height=900,
#)

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x07\x9e\x00\x00\x04e\x08\x03\x00\x00\x00Vz\x9b\x03\x…

In [6]:
class Joining_elements:
 
    def __init__(self):
        self.engine = sa.create_engine(f'sqlite:///{folder_path}/database/pagilla.db', echo=False)
        self.conn = self.engine.connect()
        self.tables = self.get_tables()
        self.table_columns = self.get_table_details()
        
        self.valid_joins = ['LEFT JOIN', 'RIGHT JOIN', 'INNER JOIN', 'CROSS JOIN']
        
        self.left_table = self.tables[0]
        self.right_table = self.tables[0]
        self.left_column = self.table_columns[self.left_table][0]
        self.right_column = self.table_columns[self.right_table][0]
        self.join_type = self.valid_joins[0]
        
        self.left_table_df = self.get_table(self.left_table)
        self.right_table_df = self.get_table(self.right_table)
        self.join_table_df, self.query_SQL = self.get_join_table()
        
    def get_tables(self):
        inspector = sa.inspect(self.engine)
        tables = inspector.get_table_names()
        return tables
    
    def get_table_details(self):
        m = sa.MetaData()
        m.reflect(self.engine)
        table_columns = {name: table.columns.keys() for name, table in m.tables.items()}
        return table_columns
    
    def get_table(self, table_name):
        table = pd.read_sql(f'SELECT * FROM {table_name}', con=self.engine)
        return table
    
    def get_join_table(self): 
        join_type = self.join_type
        if join_type == 'CROSS JOIN':
            sql = f"SELECT * FROM {self.left_table} as l {join_type} {self.right_table} as r"
        else:
            sql = f"SELECT * FROM {self.left_table} as l {join_type} {self.right_table} as r ON l.{self.left_column} = r.{self.right_column}"
        #print(s)
        join_table = pd.read_sql(sql, con=self.engine)
        #join_table = pd.read_sql(f"SELECT * FROM {left} as l {join} {right} as r ON l.{on_left} = r.{on_right}", con=self.engine)
        return join_table, sql
    
    def update_left_table(self, table_name):
        if table_name in self.tables:
            self.left_table = table_name
            table_df = self.get_table(table_name)
            self.left_table_df = table_df
            table_columns = self.table_columns[table_name]
            self.left_column = table_columns[0]
            self.join_table_df, self.query_SQL = self.get_join_table()
            
            return table_df, table_columns, self.left_column, self.join_table_df, self.query_SQL
        else:
            return pd.DataFrame(), [], '', pd.DataFrame(), ''
        
    def update_right_table(self, table_name):
        if table_name in self.tables:
            self.right_table = table_name
            table_df = self.get_table(table_name)
            self.right_table_df = table_df
            table_columns = self.table_columns[table_name]
            self.right_column = table_columns[0]
            self.join_table_df, self.query_SQL = self.get_join_table()
            
            return table_df, table_columns, self.right_column, self.join_table_df, self.query_SQL
        else:
            return pd.DataFrame(), [], '', pd.DataFrame(), ''
        
    def update_left_column(self, column_name):
        possible_columns = self.table_columns[self.left_table]
        if column_name in possible_columns:
            self.left_column = column_name
            self.join_table_df, self.query_SQL = self.get_join_table()
            
            return self.join_table_df, self.query_SQL
        else:
            return pd.DataFrame(), ''
        
    def update_right_column(self, column_name):
        possible_columns = self.table_columns[self.right_table]
        if column_name in possible_columns:
            self.right_column = column_name
            self.join_table_df, self.query_SQL = self.get_join_table()
            
            return self.join_table_df, self.query_SQL
        else:
            return pd.DataFrame(), ''
    
    def update_join_type(self, join_name):
        if join_name in self.valid_joins:
            self.join_type = join_name
            self.join_table_df, self.query_SQL = self.get_join_table()
        
            return self.join_table_df, self.query_SQL
        else:
            return pd.DataFrame(), ''
            
            
    def __del__(self):
        self.conn.close()

In [7]:
data_tables = Joining_elements()

In [8]:
left_drop = widgets.Dropdown(options=data_tables.tables,
                             value=data_tables.left_table,
                             description='Left table:',
                             disabled=False,
                            )
right_drop = widgets.Dropdown(options=data_tables.tables,
                              value=data_tables.right_table,
                              description='Right table:',
                              disabled=False,
                             )
left_columns_drop = widgets.Dropdown(options=data_tables.table_columns[left_drop.value],
                                     value=data_tables.left_column,
                             description='Left columns:',
                             disabled=False,
                            )
right_columns_drop = widgets.Dropdown(options=data_tables.table_columns[right_drop.value],
                                      value=data_tables.right_column,
                             description='Right columns:',
                             disabled=False,
                            )

In [17]:
join_type_drop = widgets.Dropdown(options=data_tables.valid_joins,
                                  value=data_tables.join_type,
                                  description='Join type:',
                                  disabled=False,
                                 )
query_text = widgets.Label(value=data_tables.query_SQL,
                                 #placeholder='',
                                 #description='',
                                 #width='100%',
                                 disabled=True   
                                )
right_text = widgets.Text(value="  ",
                                  placeholder='',
                                  description='',
                                  disabled=True   
                                 )

In [18]:
#pd.read_sql(f"SELECT * FROM actor as l CROSS JOIN address as r", con=engine)

In [19]:
#left_table, right_table, join_table = data_tables(engine, left_drop.value, right_drop.value, join_drop.value, join_type_drop.value)
out_left = widgets.Output()
with out_left:
        display(data_tables.left_table_df, clear=True)
        
out_right = widgets.Output()
with out_right:
        display(data_tables.right_table_df, clear=True)
        
out_join = widgets.Output()
with out_join:
        display(data_tables.join_table_df, clear=True)

In [20]:
def left_table_change(change):
    left_table, left_columns_drop.options, left_columns_drop.value, join_table, query_text.value = data_tables.update_left_table(change.new)
    
    with out_left:
        display(left_table, clear=True)
    with out_join:
        display(join_table, clear=True)
        
def right_table_change(change):
    right_table, right_columns_drop.options, right_columns_drop.value, join_table, query_text.value = data_tables.update_right_table(change.new)
    with out_right:
        display(right_table, clear=True)
    with out_join:
        display(join_table, clear=True)
        
def left_column_change(change):
    join_table, query_text.value = data_tables.update_left_column(change.new)
    with out_join:
        display(join_table, clear=True)
        
def right_column_change(change):
    join_table, query_text.value = data_tables.update_right_column(change.new)
    with out_join:
        display(join_table, clear=True)
    
def join_change(change):
    join_table, query_text.value = data_tables.update_join_type(change.new)
    with out_join:
        display(join_table, clear=True)


left_drop.observe(left_table_change, names='value')
right_drop.observe(right_table_change, names='value')
left_columns_drop.observe(left_column_change, names='value')
right_columns_drop.observe(right_column_change, names='value')
join_type_drop.observe(join_change, names='value')

In [24]:
#display(, )
#display(widgets.VBox([widgets.HBox([left_drop, left_columns_drop]),
#                      widgets.HBox([right_drop, right_columns_drop]),
#                      join_type_drop]))

In [22]:
#widgets.VBox([left_drop, left_columns_drop, out_left, right_drop, right_columns_drop, out_right, join_type_drop, query_text, out_join])

In [23]:
widgets.HBox([widgets.VBox([left_drop, left_columns_drop, out_left]),
              widgets.VBox([join_type_drop, query_text, out_join]),
              widgets.VBox([right_drop, right_columns_drop, out_right]),
             ])